# Header

In [1]:
import pandas as pd
import time
import numpy as np

# Metrics
from sklearn.metrics import accuracy_score, f1_score

In [2]:
from data_treatment import train_df, test_df, \
                            treated_train_df, treated_test_df, \
                            new_features_train_df, new_features_test_df, \
                            svd_train_df, svd_test_df, \
                            targets_for_test_df

from data_treatment import train_val_split

In [3]:
# Define function to create sequences
def create_sequences(data, num_timesteps):
    sequences = []
    for i in range(len(data) - num_timesteps + 1):
        sequences.append(data[i:i+num_timesteps])
    return np.array(sequences)


# Base RNN

## Crude

In [4]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense
from sklearn.preprocessing import StandardScaler
import tensorflow_addons as tfa


start_time = time.time()


# Extract features and target
X_train, y_train, X_val, y_val = train_val_split(train_df)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)



# Reshape data to fit RNN input requirements (samples, time steps, features)
num_timesteps = 15
num_features = X_train.shape[1]  # Assuming X_train has 10 features

# Generate sequences
X_train_seq = create_sequences(X_train, num_timesteps)
X_val_seq = create_sequences(X_val, num_timesteps)

# Adjust y_train and y_val accordingly
y_train_seq = y_train[num_timesteps - 1:]
y_val_seq = y_val[num_timesteps - 1:]

# Define the RNN model
model = Sequential()
model.add(SimpleRNN(50, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tfa.metrics.F1Score(num_classes=1, threshold=0.5)])

# Train the model
model.fit(X_train_seq, y_train_seq, epochs=10, batch_size=32)

# Predict probabilities
y_pred_prob = model.predict(X_val_seq)

# Convert probabilities to binary predictions
y_pred = (y_pred_prob > 0.5).astype(int)


end_time = time.time()
minutes = (end_time - start_time) // 60
seconds = (end_time - start_time) % 60
print(f'Time elapsed: {minutes:.0f}m {seconds:.1f}s')
print('--------------------------------------')
# Calculate accuracy
accuracy = accuracy_score(y_val_seq, y_pred)
print(f'Validation Accuracy: {accuracy:.5f}')

# Calculate F1 macro score
f1_macro = f1_score(y_val_seq, y_pred, average='macro')
print(f'Validation F1 Macro Score: {f1_macro:.5f}')


C:\Users\PLour\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\PLour\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.10.0 and strictly below 2.13.0 (nightly versions are not supported). 
 The versions of Tens

ImportError: cannot import name 'tf_utils' from 'keras.utils' (C:\Users\PLour\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\api\utils\__init__.py)

In [ ]:
len(test_predictions), len(targets_for_test_df)

(909620, 909616)

In [ ]:
# Ensure the test data is preprocessed in the same way as the training data
X_test = test_df.drop(columns=['row_id'])
X_test = scaler.transform(X_test)

# Reshape test data to fit RNN input requirements (samples, time steps, features)
X_test_seq = create_sequences(X_test, num_timesteps)

# Make predictions on the test data
test_predictions_prob = model.predict(X_test_seq)

# Convert probabilities to binary predictions
test_predictions = (test_predictions_prob > 0.5).astype(int)

# Adjust test_predictions to match the length of targets_for_test_df
# filled_test_predictions = (2+(num_timesteps)) * [0] + test_predictions.flatten().tolist()

# # Calculate accuracy
# accuracy = accuracy_score(targets_for_test_df, test_predictions)
# print(f'Test Accuracy: {accuracy:.5f}')

# # Calculate F1 macro score
# f1_macro = f1_score(targets_for_test_df, test_predictions, average='macro')
# print(f'Test F1 Macro Score: {f1_macro:.5f}')

## New Features

In [ ]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense
from sklearn.preprocessing import StandardScaler
import tensorflow_addons as tfa


start_time = time.time()


# Extract features and target
X_train, y_train, X_val, y_val = train_val_split(treated_train_df)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)



# Reshape data to fit RNN input requirements (samples, time steps, features)
num_timesteps = 15
num_features = X_train.shape[1]  # Assuming X_train has 10 features

# Generate sequences
X_train_seq = create_sequences(X_train, num_timesteps)
X_val_seq = create_sequences(X_val, num_timesteps)

# Adjust y_train and y_val accordingly
y_train_seq = y_train[num_timesteps - 1:]
y_val_seq = y_val[num_timesteps - 1:]

# Define the RNN treated_model
treated_model = Sequential()
treated_model.add(SimpleRNN(50, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
treated_model.add(Dense(1, activation='sigmoid'))
treated_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tfa.metrics.F1Score(num_classes=1, threshold=0.5)])

# Train the treated_model
treated_model.fit(X_train_seq, y_train_seq, epochs=10, batch_size=32)

# Predict probabilities
y_pred_prob = treated_model.predict(X_val_seq)

# Convert probabilities to binary predictions
y_pred = (y_pred_prob > 0.5).astype(int)


end_time = time.time()
minutes = (end_time - start_time) // 60
seconds = (end_time - start_time) % 60
print(f'Time elapsed: {minutes:.0f}m {seconds:.1f}s')
print('--------------------------------------')
# Calculate accuracy
accuracy = accuracy_score(y_val_seq, y_pred)
print(f'Validation Accuracy: {accuracy:.5f}')

# Calculate F1 macro score
f1_macro = f1_score(y_val_seq, y_pred, average='macro')
print(f'Validation F1 Macro Score: {f1_macro:.5f}')


In [ ]:
# Ensure the test data is preprocessed in the same way as the training data
X_test = treated_test_df.drop(columns=['row_id'])
X_test = scaler.transform(X_test)

# Reshape test data to fit RNN input requirements (samples, time steps, features)
X_test_seq = create_sequences(X_test, num_timesteps)

# Make predictions on the test data
test_predictions_prob = treated_model.predict(X_test_seq)

# Convert probabilities to binary predictions
test_predictions = (test_predictions_prob > 0.5).astype(int)

# Adjust test_predictions to match the length of targets_for_test_df
# filled_test_predictions = (2+(num_timesteps)) * [0] + test_predictions.flatten().tolist()

# # Calculate accuracy
# accuracy = accuracy_score(targets_for_test_df, test_predictions)
# print(f'Test Accuracy: {accuracy:.5f}')

# # Calculate F1 macro score
# f1_macro = f1_score(targets_for_test_df, test_predictions, average='macro')
# print(f'Test F1 Macro Score: {f1_macro:.5f}')

## Only new features

In [ ]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense
from sklearn.preprocessing import StandardScaler
import tensorflow_addons as tfa


start_time = time.time()


# Extract features and target
X_train, y_train, X_val, y_val = train_val_split(new_features_train_df)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)



# Reshape data to fit RNN input requirements (samples, time steps, features)
num_timesteps = 15
num_features = X_train.shape[1]  # Assuming X_train has 10 features

# Generate sequences
X_train_seq = create_sequences(X_train, num_timesteps)
X_val_seq = create_sequences(X_val, num_timesteps)

# Adjust y_train and y_val accordingly
y_train_seq = y_train[num_timesteps - 1:]
y_val_seq = y_val[num_timesteps - 1:]

# Define the RNN only_model
only_model = Sequential()
only_model.add(SimpleRNN(50, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
only_model.add(Dense(1, activation='sigmoid'))
only_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tfa.metrics.F1Score(num_classes=1, threshold=0.5)])

# Train the only_model
only_model.fit(X_train_seq, y_train_seq, epochs=10, batch_size=32)

# Predict probabilities
y_pred_prob = only_model.predict(X_val_seq)

# Convert probabilities to binary predictions
y_pred = (y_pred_prob > 0.5).astype(int)


end_time = time.time()
minutes = (end_time - start_time) // 60
seconds = (end_time - start_time) % 60
print(f'Time elapsed: {minutes:.0f}m {seconds:.1f}s')
print('--------------------------------------')
# Calculate accuracy
accuracy = accuracy_score(y_val_seq, y_pred)
print(f'Validation Accuracy: {accuracy:.5f}')

# Calculate F1 macro score
f1_macro = f1_score(y_val_seq, y_pred, average='macro')
print(f'Validation F1 Macro Score: {f1_macro:.5f}')


array([[1],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [ ]:
# Ensure the test data is preprocessed in the same way as the training data
X_test = new_features_test_df.drop(columns=['row_id'])
X_test = scaler.transform(X_test)

# Reshape test data to fit RNN input requirements (samples, time steps, features)
X_test_seq = create_sequences(X_test, num_timesteps)

# Make predictions on the test data
test_predictions_prob = only_model.predict(X_test_seq)

# Convert probabilities to binary predictions
test_predictions = (test_predictions_prob > 0.5).astype(int)

# Adjust test_predictions to match the length of targets_for_test_df
# filled_test_predictions = (2+(num_timesteps)) * [0] + test_predictions.flatten().tolist()

# # Calculate accuracy
# accuracy = accuracy_score(targets_for_test_df, test_predictions)
# print(f'Test Accuracy: {accuracy:.5f}')

# # Calculate F1 macro score
# f1_macro = f1_score(targets_for_test_df, test_predictions, average='macro')
# print(f'Test F1 Macro Score: {f1_macro:.5f}')